In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nucls-classes/nucleus_GTcodes.csv


In [2]:
#check gpu
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [3]:
#for Downloading from gdrive
! pip install gdown

In [19]:
import gdown
url = "https://drive.google.com/file/d/11ZedPkgI3YB-1_Nca1OUMt1eDc4JbUXD/view?usp=share_link"

In [20]:
#download our dataset
gdown.download(url,"QC.zip", quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=11ZedPkgI3YB-1_Nca1OUMt1eDc4JbUXD
To: /kaggle/working/QC.zip
100%|██████████| 805M/805M [00:08<00:00, 95.9MB/s] 


'QC.zip'

In [8]:
#unzip our dataset
!unzip QC.zip

Archive:  QC.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of QC.zip or
        QC.zip.zip, and cannot find QC.zip.ZIP, period.


In [9]:
#we dont need zip file anymore
!rm QC.zip

In [10]:
#install neccessary libraries
!pip install datasets evaluate transformers accelerate pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 292.8 kB/s eta 0:00:00a 0:00:01


In [13]:
from datasets import load_dataset,concatenate_datasets
import numpy as np

In [14]:
#checking is cuda enable
import torch 
torch.cuda.is_available()

False

In [15]:
#checking python version
!python -V

Python 3.7.12


In [16]:
#loading our dataset using load_dataset function
dataset = load_dataset("imagefolder", data_dir="QC/rgb")
#in python 3.7  load_dataset function automatically add label. since we use it just for load our images we are removing
dataset = dataset.remove_columns(["label"])

FileNotFoundError: Unable to resolve any data file that matches '['QC/rgb/**']' at /kaggle/working

In [ ]:
#we are loading our masks 
mask_dataset = load_dataset("imagefolder", data_dir="QC/mask")
mask_dataset = mask_dataset.remove_columns(["label"])

In [ ]:
#!rm -rf QC/

In [ ]:
#this is example image
dataset['train'][0]['image']

In [ ]:
#mask image of same example on the above
mask_dataset['train'][0]['image']

In [ ]:
#we are checking our dataset object
dataset

In [ ]:
#since in tutorial they rename images as pixel_values we are also renaming
dataset['train'] = dataset['train'].rename_column("image", "pixel_values")

In [ ]:
#checking our Dataset object again
dataset['train']

In [17]:
#in the tutorial they also rename masks as label so we are rename it.
mask_dataset['train']= mask_dataset['train'].rename_column("image", "label")

NameError: name 'mask_dataset' is not defined

In [18]:
mask_dataset['train']

NameError: name 'mask_dataset' is not defined

In [23]:
#we are preparing 2 function. applyGrayScaleResize is convert our mask images into grayscale images and resize it to 200x200
def applyGrayScaleResize(data):
    data['label']=[image.convert('L').resize((200,200)) for image in data["label"]]
    return data
#in dsresize is resizeing our images into 200x200
def dsresize(data):
    data['pixel_values']=[image.resize((200,200)) for image in data["pixel_values"]]
    return data
#we are applying our functions on both dataset object. sometime it takes time
mask_dataset=mask_dataset.map(applyGrayScaleResize, batched=True)
dataset=dataset.map(dsresize, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [24]:
mask_dataset['train']['label']

[<PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512>,
 <PIL.Pn

In [25]:
# we add this cell in order to find how many different color there are in masks. Our model expect classes in different color in mask.
# we were getting error caused of class numbers are not matching so add this and use lengh as class number
def get_unique_colors(images):
    # Create an empty set to store unique colors
    unique_colors = set()
    for im in images:
        
        # Convert image to RGB mode
        rgb_im = im.convert('RGB')
        # Iterate over pixels in the image
        for r, g, b in rgb_im.getdata():
            # Add the RGB tuple to the set
            unique_colors.add((r, g, b))
    # Return the list of unique colors
    return list(unique_colors)
colors=get_unique_colors(mask_dataset['train']['label'])
print(len(colors))

94


In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pixel_values'],
        num_rows: 1744
    })
})

In [27]:
#for merging our mask_dataset and dataset we are creating empty DatasetDict object
from datasets.io.abc import DatasetDict
ds=DatasetDict()

In [28]:
#we are merging all of our images and masks in ds
ds['train']=concatenate_datasets([dataset['train'],mask_dataset['train']],axis=1)

In [29]:
#since we dont need these object we are deleting
del dataset
del mask_dataset

In [12]:
#we are splitting test set in order to do validation
ds=ds['train'].train_test_split(test_size=0.2)

NameError: name 'ds' is not defined

In [31]:
# we are checking our ds object
ds

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'label'],
        num_rows: 1395
    })
    test: Dataset({
        features: ['pixel_values', 'label'],
        num_rows: 349
    })
})

In [33]:
train_ds = ds["train"]
test_ds = ds["test"]

In [34]:
#We are defining our feature extractor methods for segformer. using this process our image data for segformer format
from transformers import (
    SegformerFeatureExtractor,
)

feature_extractor = SegformerFeatureExtractor()
def train_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [35]:
# we were reading our original classes provided with dataset. but since our detected colors doesnt match we are not using.
#our plan is after training analyze results and downgrade detected class to  original classes
#classes=pd.read_csv('nucleus_GTcodes.csv')

In [39]:
#transformers models expecting classes in key value format. in id2label keys are index and values are class names and label2id keys are class names and values are index
#classList=classes['group'].tolist()

#id2label={v: k for v, k in enumerate(classList)}
#label2id={k: v for v, k in enumerate(classList)}

In [40]:
#since we detected how many classes we have in masks and doesnt know which color is what we are creating dummy id2label and label2id in order to do training
id2label={}
label2id={}
for i in range(len(colors)):
    id2label[i]=i
    label2id[i]=i

In [41]:
#id2label

In [42]:
#label2id

In [43]:
train_ds

Dataset({
    features: ['pixel_values', 'label'],
    num_rows: 1395
})

In [44]:
#we are downloading our pretrained model.
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
    , low_cpu_mem_usage=True
)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_

In [45]:
#we are defining training arguments. 
from transformers import TrainingArguments

epochs = 1
lr = 0.00006
batch_size = 2

training_args = TrainingArguments(
    "test",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    report_to="none"
)

In [46]:
#in kaggle we are doing this since we dont want to use wandb
import os
os.environ["WANDB_DISABLED"] = "True"

In [47]:
#this cell provided by huggingface. It is used for calculating accuray and iou 
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
        logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        # currently using _compute instead of compute
        # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
        metrics = metric._compute(
                predictions=pred_labels,
                references=labels,
                num_labels=len(id2label),
                ignore_index=0,
                reduce_labels=feature_extractor.reduce_labels,
            )
    
    # add per category metrics as individual key-value pairs
        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

In [48]:
# we are defining our trainer object for train our model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    
)

In [ ]:
#we are starting our model train
#In training due to ram size notebook always crashing. Because of this I cant continue. I tried do alternative they suggest like deepspeed. but I couldnt succeed.
#if training was succeed I was planning to use segmentation pipelines for make inference.
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1395
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 44
/opt/conda/lib/python3.7/site-packages/transformers/data/data_collator.py:131: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  batch[k] = torch.tensor([f[k] for f in features])
/opt/conda/lib/python3.7/site-pac

Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 349
  Batch size = 32
